# 11. 뉴스 요약봇 만들기
## 11-2. 텍스트 요약(Text Summarization)
- 긴 길이의 문서(Document) 원문을 핵심 주제만으로 구성된 짧은 요약(Summary) 문장들로 변환하는 것

```text
ex) 뉴스기사 (큰 텍스트) --> 뉴스 제목 (작은 텍스트)

중요한점: 요약 전후에 정보 손실 발생이 최소화되어야 한다.
```

요약 문장을 만드는 방법?</br>
1. 추출적 요약(Extractive Summarization)
    - 원문에서 **문장들을 추출**해서 요약하는 방식
    - 딥러닝 보다는 주로 전통적인 머신러닝 방식에서 사용
        - 텍스트 랭크(Text Rank)
            - 각 문장에 중요도(랭크)를 설정하여 중요한 문장 몇 가지를 추출하는 방식
        - 대표적인 사용처
            - 네이버 뉴스 서비스에 있는 `요약봇`
                - 요약문에 사용된 문장 3개가 원문에 그대로 있는것이 특징
2. 추상적 요약(Abstractive Summarization)
    - 원문으로부터 내용이 요약된 **새로운 문장을 생성**
        - 새로운 문장은 원문에 원래 없던 문장일 수도 있다.
    - 자연어 처리 분야 중 하나를 사용
        - 자연어 생성(Natural Language Generation, NLG) 기법
    - 원문을 구성하는 문장 중 어느 것이 요약문에 들어갈 핵심문장인지를 판별한다는 점
        - 문장 분류(Text Classification) 문제로 볼 수도 있다.
    - 기본적인 신경망 접근법
        - RNN
            - 먼 과거의 정보를 현재에 전달하기 어려운 `장기 의존성(Long Term Dependencies)` 문제가 있다.
    - 구글의 텍스트 요약 모델의 문장 요약법
        - 역문서빈도(IDF) 지표
            - 문서 안에서 중요해 보이는 부분을 추출하여 요약문에 담는 방식
            - 이런 발췌 방식은 어색하거나 문법적으로 이상한 결과물을 만들 수 있다. (ex: `앨리스와 밥은 동물원에 가다. 조류 한 무리를 봤다.`)
        - 추상적 요약(Abstractive Summarization)
            - ex: `앨리스와 밥은 동물원에 갔고 짐승과 새를 봤다.`
    - seq2seq
        - 짧은 문장, 요약문을 생성하는 구글의 모델
        - 딥러닝을 통해 `end-to-end`로 설계

## 11-3. 인공 신경망으로 텍스트 요약 훈련시키기
**seq2seq 개요**</br>
![Summary of seq2seq](../images/23.png)</br>
[Link](https://medium.com/dl-for-product-and-service/abstractive-text-summary-with-reinforcement-learning-ab2458ab29d5)</br>

원문을 첫 번째 RNN인 encoder로 입력하면 encoder는 이를 하나의 고정된 벡터로 변환</br>
이 벡터를 문맥 정보를 가진 벡터; `컨택스트 벡터(Context vector)`라고 한다.</br>

두 번째 RNN인 decoder는 이 컨택스트 벡터를 전달받아 한 단어씩 생성하면서 요약 문장을 완성</nr>

**LSTM과 컨텍스트 벡터**</br>
seq2seq를 구현할 때, 인코더/디코더로 vanilla RNN이 아니고 LSTM을 사용할 것임.</br>
![RNN과 LSTM](../images/24.png)</br>
[Link](https://colah.github.io/posts/2015-08-Understanding-LSTMs)</br>

LSTM은 다음 time step의 셀에 `hidden state`와 `cell state`도 함께 전달한다.</br>
(Vanilla RNN과 다른점)</br>

**시작 토큰과 종료 토큰**</br>
![시작 토큰 SOS와 종료 토큰 EOS는 각각 start of a sequence와 end of a sequnce를 나타낸다.](../images/25.png)</br>
[Link](https://arxiv.org/pdf/1812.02303.pdf)</br>

seq2seq 구조에서 디코더는 시작 토큰 SOS가 입력되면, 각 시점마다 단어를 생성하고 이 과정을 종료 토큰 EOS를 예측하는 순간 멈추지 않는다.</br>
즉 시작 토큰과 종료 토큰을 넣어줄 때 (`a.k.a 전처리`) 어디서 멈춰야 할지를 알려줘야 한다.</br>

**어텐션 메커니즘을 통한 새로운 컨텍스트 벡터 사용하기**</br>
![기존의 컨텍스트 벡터보다 인코더의 정보를 적극적으로 활용하여 성능을 끌어올리는 어텐션 메커니즘](../images/26.png)</br>
[Link](https://arxiv.org/pdf/1812.02303.pdf)</br>

기존의 seq2seqe는 **인코더의 마지막 time step의 hidden state**를 컨택스트 벡터로 사용하였다.</br>
 - RNN 계열의 인공 신경망 (RNN, LSTM, GRU)의 한계로 인해 이 컨택스트 정보에는 이미 입력 시퀀스의 많은 정보가 손실이 된 상태다.

**어텐션 메커니즘(Attention Mechanism)**은 이와 달리, **인코더의 모든 step의 hidden state의 정보**가 컨텍스트 벡터에 전부 반영되도록 한다.</br>
 - hidden state는 모든 인코더에 동일한 비중으로 반영되지 않는다.
 - 디코더의 현재 time step의 예측 + 인코더의 각 step이 얼마나 영향을 미치는지에 따른 가중합으로 계산되는 방식

주의점</br>
- 컨텍스트 백터를 구성하기 위한 인코더의 hidden state의 가중치 값은 `디코더의 현재 스텝에 따라 계속 달라진다.`
    - 디코더의 현재 문장 생성 부위가 주어부인지 술어부인지 목적어인지 등에 따라 인코더가 입력 데이터를 해석한 컨텍스트 벡터가 다른 값이 된다.

## 11-4. 데이터 준비하기
작업환경 구성(노드 기준)</br>
```shell
$ mkdir -p ~/aiffel/news_summarization/data
```

사용할 데이터셋은 Kaggle에서 제공된 `아마존 리뷰 데이터셋`</br>
[Link: Amazon Fine Food Revies](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)</br>

클라우드 데이터셋 이용을 위한 추가 작업
```shell
$ ln -s ~/data/*.csv ~/aiffel/news_summarization/data
```

이번 실습에서는 NLTK의 불용어(stopwords)를 사용함
 - NLTK: Natural Language Toolkit
 - 영어 기호, 통계, 자연어 처리를 위한 라이브러리
 - 기본적으로 100여 개의 불용어를 가지고 있음

In [ ]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as np
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

print('=3')

```shell
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip


=3
```

링크에서 다운받은 데이터(`Reviews.csv`)는 총 568,454개의 샘플을 가지고 있다.</br>
테스트에서는 10만개만을 사용한다.

In [ ]:
data = pd.read_csv(os.getenv("HOME")+"/aiffel/news_summarization/data/Reviews.csv", nrows=100000)
print(f"전체 샘플수: {len(data)}")

```shell
전체 샘플수 : 100000
```

전체 결과 중 5개만 출력</br>

```python
data.head()
```

![Dataframe head](../images/27.png)</br>

전체 데이터 중 Summary, Text 열들만 별도로 저장 후 출력해보기

In [ ]:
# input [[YOUR CODE]] aside `data`
data = data[['Text', 'Summary']]
data.head()

# 랜덤한 15개 샘플 출력
data.sample(15)

![15 random samples, Text & Summary cols](../images/28.png)</br>

인공 신경망을 통해 `Text`를 입력받으면 `Summary`를 예측하도록 인공 신경망을 훈련시킬 것이다.

## 11-5. 데이터 전처리하기
### (1) 데이터 정리하기

